In [1]:
cd ..

d:\research\fedmiss_bench


In [27]:
import os
import itertools
import json
import pandas as pd

## Load Evaluation Results

In [45]:
nnn_imp_fed = {
    'simple': ['local', 'fedavg', 'central'],
    'em': ['local', 'fedavg', 'central'],
    'linear_ice': ['local', 'fedavg', 'central'],
    'missforest': ['local', 'fedtree', 'central']
}

nn_imp_fed = {
    'miwae': ['local', 'fedavg', 'fedavg_ft', 'fedprox', 'central'],
    'gain': ['local', 'fedavg', 'fedavg_ft', 'fedprox', 'central'],
}

dataset_dp = {
    'codrna': ['iid-even', 'iid-uneven', 'niid-f1', 'niid-f2'],
    'california': ['iid-even', 'iid-uneven', 'niid-t1', 'niid-t2'],
    'hhip': ['iid-even', 'iid-uneven', 'niid-t1', 'niid-t2'],
    'dvisits': ['iid-even', 'iid-uneven', 'niid-t1', 'niid-t2'],
    'vehicle': ['iid-even', 'iid-uneven', 'niid-f1', 'niid-f2'],
    'school_pca': ['iid-even', 'iid-uneven', 'niid-f1', 'niid-f2'],
}

dataset_metric_pred = {
    'codrna': ['accuracy', 'f1', 'auc', 'prc'],
    'california': ['mse', 'mae', 'r2'],
    'hhip': ['mse', 'mae', 'r2'],
    'dvisits': ['mse', 'mae', 'r2'],
    'vehicle': ['mse', 'mae', 'r2'],
    'school_pca': ['mse', 'mae', 'r2'],
}

In [55]:
def load_ret(dataset, imputers, met):
    
    data_partition = dataset_dp[dataset]
    missing = ['mcar', 'mar-homog', 'mnar1-homog', 'mnar2-homog', 'mar-heter', 'mnar1-heter', 'mnar2-heter']
    models = ['linear', 'nn']
    rets = []
    
    for imputer in imputers:
        
        if imputer in nnn_imp_fed:
            rounds = [0,1,2,3,4,5,6,7,8,9]
            feds = nnn_imp_fed[imputer]
        else:
            rounds = [0,1,2,3,4]
            feds = nn_imp_fed[imputer]

        param = itertools.product(feds, data_partition, missing, rounds, models)
        dir_path = f'./results/raw_results/eval_results/alpha/{dataset}/'

        unfinished = set()
        count = 0
        for i, (fed, dp, ms, round, model) in enumerate(param):
            
            file_path = f'{dir_path}{dp}_{ms}/{imputer}/{fed}/{round}/{model}/eval_results.json'
        
            try:
                with open(file_path, 'r') as f:
                    ret = json.load(f)
                    if met == 'imp':
                        metrics = ['rmse', 'sliced-ws']
                        ret_metrics = [ret['results']['imp_quality'][metric] for metric in metrics]
                    elif met == 'pred':
                        metrics = dataset_metric_pred[dataset]
                        ret_metrics = [ret['results']['pred_performance'][metric] for metric in metrics]
                    else:
                        raise ValueError('Invalid metric')
                    
                    for id, items in enumerate(zip(*ret_metrics)):
                        setting = {
                            'dataset': dataset,
                            'data_partition': dp,
                            'imputer': imputer,
                            'fed': fed,
                            'missing': ms,
                            'round': round,
                            'model': model,
                            'client_id': id
                        }
                        setting.update({metric: item for metric, item in zip(metrics, items)})
                        rets.append(setting)
            except Exception as e:
                print(file_path)
                count += 1
                unfinished.add((dp, ms, imputer, fed))
                continue

        print(count)
        
    return rets, unfinished

In [56]:
result_pred, unfinished = load_ret('codrna', ['simple', 'em', 'linear_ice', 'miwae', 'gain'], met = 'pred')

0
0
0
0
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar1-homog/gain/local/4/linear/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar1-homog/gain/local/4/nn/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar2-homog/gain/local/4/linear/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar2-homog/gain/local/4/nn/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar1-homog/gain/fedavg/4/linear/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar1-homog/gain/fedavg/4/nn/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar2-homog/gain/fedavg/4/linear/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar2-homog/gain/fedavg/4/nn/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar1-homog/gain/fedavg_ft/4/linear/eval_results.json
./results/raw_resul

In [57]:
result_imp, unfinished = load_ret('codrna', ['simple', 'em', 'linear_ice', 'miwae', 'gain'], met = 'imp')

0
0
0
0
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar1-homog/gain/local/4/linear/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar1-homog/gain/local/4/nn/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar2-homog/gain/local/4/linear/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar2-homog/gain/local/4/nn/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar1-homog/gain/fedavg/4/linear/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar1-homog/gain/fedavg/4/nn/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar2-homog/gain/fedavg/4/linear/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar2-homog/gain/fedavg/4/nn/eval_results.json
./results/raw_results/eval_results/alpha/codrna/iid-uneven_mnar1-homog/gain/fedavg_ft/4/linear/eval_results.json
./results/raw_resul

## Improvements Imputation Qaulity

In [68]:
def calculate_improvement(df):
    columns={item: f'{item}_local' for item in ['rmse', 'sliced-ws']}
    df = df.copy()
    local_baseline = df[df['fed'] == 'local'].copy()
    local_baseline.rename(columns=columns, inplace=True)
    keys = ['dataset', 'data_partition', 'missing', 'imputer', 'client_id', 'round']
    df = df.merge(local_baseline[keys + list(columns.values())], on=keys, how='left')
    
    for item in ['rmse', 'sliced-ws']:
        df[f'{item}_diff'] = df[item] - df[f'{item}_local']
        df[f'{item}_improv'] = (df[f'{item}_diff'] / df[f'{item}_local']) * 100
    
    df = df[df['fed'] != 'local']
    #final_df = df[['dataset', 'partition', 'missing', 'imp', 'fed', 'client_id', 'round_id', 'rmse_diff', 'rmse_improv', 'ws_diff', 'ws_improv']]
    
    return df

In [74]:
df_imp = pd.DataFrame(result_imp)
df_imp = df_imp[df_imp['model'] != 'linear']
df_imp = df_imp.drop(columns=['model'])
df_imp_improv = calculate_improvement(df_imp)

## Improvements Downstream Performance

In [79]:
def calculate_improvement(df, dataset_name):
    columns={item: f'{item}_local' for item in dataset_metric_pred[dataset_name]}
    df = df.copy()
    local_baseline = df[df['fed'] == 'local'].copy()
    local_baseline.rename(columns=columns, inplace=True)
    keys = ['dataset', 'data_partition', 'missing', 'imputer', 'client_id', 'round']
    df = df.merge(local_baseline[keys + list(columns.values())], on=keys, how='left')
    
    for item in dataset_metric_pred[dataset_name]:
        df[f'{item}_diff'] = df[item] - df[f'{item}_local']
        df[f'{item}_improv'] = (df[f'{item}_diff'] / df[f'{item}_local']) * 100
    
    df = df[df['fed'] != 'local']
    #final_df = df[['dataset', 'partition', 'missing', 'imp', 'fed', 'client_id', 'round_id', 'rmse_diff', 'rmse_improv', 'ws_diff', 'ws_improv']]
    
    return df

In [80]:
df_pred = pd.DataFrame(result_pred)
df_pred = df_pred[df_pred['model'] != 'linear']
df_pred = df_pred.drop(columns=['model'])
df_pred_improv = calculate_improvement(df_pred, 'codrna')
df_pred_improv

,dataset,data_partition,imputer,fed,missing,round,client_id,accuracy,f1,auc,...,auc_local,prc_local,accuracy_diff,accuracy_improv,f1_diff,f1_improv,auc_diff,auc_improv,prc_diff,prc_improv
2800,codrna,iid-even,simple,fedavg,mcar,0,0,0.955556,0.929825,0.980694,...,0.980556,0.954552,0.005556,0.584795,0.009471,1.029015,0.000139,0.014164,0.000063,0.006652
2801,codrna,iid-even,simple,fedavg,mcar,0,1,0.894444,0.815534,0.984861,...,0.985833,0.971008,0.000000,0.000000,0.000000,0.000000,-0.000972,-0.098619,-0.002026,-0.208655
2802,codrna,iid-even,simple,fedavg,mcar,0,2,0.944444,0.916667,0.989861,...,0.986528,0.971069,0.005556,0.591716,0.009104,1.003086,0.003333,0.337885,0.006311,0.649874
2803,codrna,iid-even,simple,fedavg,mcar,0,3,0.911111,0.846154,0.989028,...,0.987500,0.979035,0.000000,0.000000,0.000000,0.000000,0.001528,0.154712,0.003369,0.344093
2804,codrna,iid-even,simple,fedavg,mcar,0,4,0.861111,0.747475,0.977917,...,0.982083,0.971527,-0.022222,-2.515723,-0.040404,-5.128205,-0.004167,-0.424268,-0.009697,-0.998104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39115,codrna,niid-f2,gain,central,mnar2-heter,4,5,0.813636,0.775956,0.891047,...,0.899651,0.850905,0.000000,0.000000,0.024441,3.252247,-0.008604,-0.956349,-0.089664,-10.537434
39116,codrna,niid-f2,gain,central,mnar2-heter,4,6,0.695652,0.395062,0.706635,...,0.644462,0.253237,-0.043478,-5.882353,0.072481,22.469136,0.062173,9.647345,0.007504,2.963050
39117,codrna,niid-f2,gain,central,mnar2-heter,4,7,0.861538,0.590909,0.916952,...,0.908952,0.797557,-0.038462,-4.273504,-0.120202,-16.903409,0.008000,0.880134,-0.122493,-15.358535
39118,codrna,niid-f2,gain,central,mnar2-heter,4,8,0.696629,0.330579,0.880932,...,0.758664,0.696418,-0.029963,-4.123711,-0.158932,-32.467532,0.122268,16.116196,0.096339,13.833527


## Imputers Comparison Imputation Quality

In [63]:
df_imp = pd.DataFrame(result_imp)
df_imp

,dataset,data_partition,imputer,fed,missing,round,model,client_id,rmse,sliced-ws
0,codrna,iid-even,simple,local,mcar,0,linear,0,0.170213,0.065619
1,codrna,iid-even,simple,local,mcar,0,linear,1,0.167724,0.053620
2,codrna,iid-even,simple,local,mcar,0,linear,2,0.162599,0.046534
3,codrna,iid-even,simple,local,mcar,0,linear,3,0.171930,0.064570
4,codrna,iid-even,simple,local,mcar,0,linear,4,0.169623,0.052067
...,...,...,...,...,...,...,...,...,...,...
78235,codrna,niid-f2,gain,central,mnar2-heter,4,nn,5,0.143059,0.076276
78236,codrna,niid-f2,gain,central,mnar2-heter,4,nn,6,0.197525,0.118690
78237,codrna,niid-f2,gain,central,mnar2-heter,4,nn,7,0.164777,0.066542
78238,codrna,niid-f2,gain,central,mnar2-heter,4,nn,8,0.205193,0.106524


## Imputers Comparison Downstream Performance

In [64]:
df_pred = pd.DataFrame(result_pred)
df_pred

,dataset,data_partition,imputer,fed,missing,round,model,client_id,accuracy,f1,auc,prc
0,codrna,iid-even,simple,local,mcar,0,linear,0,0.916667,0.885496,0.979167,0.951023
1,codrna,iid-even,simple,local,mcar,0,linear,1,0.888889,0.846154,0.955556,0.906473
2,codrna,iid-even,simple,local,mcar,0,linear,2,0.888889,0.855072,0.981250,0.966664
3,codrna,iid-even,simple,local,mcar,0,linear,3,0.783333,0.731034,0.920833,0.868993
4,codrna,iid-even,simple,local,mcar,0,linear,4,0.900000,0.863636,0.973194,0.948997
...,...,...,...,...,...,...,...,...,...,...,...,...
78235,codrna,niid-f2,gain,central,mnar2-heter,4,nn,5,0.813636,0.775956,0.891047,0.761242
78236,codrna,niid-f2,gain,central,mnar2-heter,4,nn,6,0.695652,0.395062,0.706635,0.260740
78237,codrna,niid-f2,gain,central,mnar2-heter,4,nn,7,0.861538,0.590909,0.916952,0.675064
78238,codrna,niid-f2,gain,central,mnar2-heter,4,nn,8,0.696629,0.330579,0.880932,0.792757
